In [ ]:
#-------------------------------------------------------------Buscar Veículo/Equipamento

import pandas as pd
tabela = pd.read_csv("Abastecimento_Caminhao.csv", delimiter=';', on_bad_lines='skip')

veiculoSearch = input("Veículo (exemplo: 'BT220'): ")
#Função=>listar informações do veículo
def listVeiculo(veiculoSearch, tabela):
    consultVeiculo = veiculoSearch
    #Verify se->veículo tá->coluna'Veículo/Equip.'
    if tabela['Veículo/Equip.'].str.contains(consultVeiculo, case=False, na=False).any():
        filtered_table = tabela[tabela['Veículo/Equip.'].str.contains(consultVeiculo, case=False, na=False)]
        display(filtered_table)
    else:
        print("Não há Veículos/Equipamentos com esse nome!")
#Call funç c/veículo forncd+tbl
listVeiculo(veiculoSearch, tabela)

In [ ]:
#--------------------------------------------------Comparação do Rendimento do Veículo, com outros! 

import pandas as pd
import plotly.express as px
tabela = pd.read_csv("Abastecimento_Caminhao.csv", delimiter=';', on_bad_lines='skip')
tabela['Vlr. Total'] = pd.to_numeric(tabela['Vlr. Total'].astype(str).str.replace(',', ''), errors='coerce')
tabela['Km Rodados'] = pd.to_numeric(tabela['Km Rodados'], errors='coerce')
tabela['Litros'] = pd.to_numeric(tabela['Litros'], errors='coerce')

veiculoData = input("Veículo (exemplo: 'BT220'): ")
def veiculo_Search_value(veiculoData, tabela):
    veiculoType = veiculoData
    if tabela['Veículo/Equip.'].str.contains(veiculoType, case=False, na=False).any():
        consultVeicul = tabela[tabela['Veículo/Equip.'].str.contains(veiculoType, case=False, na=False)].copy()
        consultVeicul['Lucro'] = consultVeicul['Vlr. Total'] - consultVeicul['Km Rodados']
        consultVeicul['Km por Litro'] = consultVeicul['Km Rodados'] / consultVeicul['Litros']
        
        resumo_selecionado = consultVeicul.agg({
            'Lucro': 'sum',
            'Km por Litro': 'mean',
            'Km Rodados': 'sum',
            'Litros': 'sum'
        })

        tabela['Lucro'] = tabela['Vlr. Total'] - tabela['Km Rodados']
        tabela['Km por Litro'] = tabela['Km Rodados'] / tabela['Litros']
        
        melhor_resultado = tabela.groupby(['Requisitante', 'Veículo/Equip.']).agg({
            'Lucro': 'sum',
            'Km por Litro': 'mean',
            'Km Rodados': 'sum',
            'Litros': 'sum'
        }).idxmax()

        melhores_requisitante, melhores_veiculo = melhor_resultado['Lucro']
        melhor_table = tabela[(tabela['Requisitante'] == melhores_requisitante) & (tabela['Veículo/Equip.'] == melhores_veiculo)].copy()
        resumo_melhor = melhor_table.agg({
            'Lucro': 'sum',
            'Km por Litro': 'mean',
            'Km Rodados': 'sum',
            'Litros': 'sum'
        })

        comparacao = pd.DataFrame({
            'Métrica': ['Lucro', 'Km por Litro', 'Km Rodados', 'Litros'],
            f'{veiculoType} - {consultVeicul["Requisitante"].iloc[0]}': resumo_selecionado.values,
            f'Melhor - {melhores_requisitante} - {melhores_veiculo}': resumo_melhor.values
        })
        fig = px.histogram(comparacao, x='Métrica', y=comparacao.columns[1:],
                     title='Comparação de Métricas',
                     barmode='group',
                     labels={'value': 'Valor', 'variable': 'Legenda'})
        fig.update_layout(xaxis_title='Métrica', yaxis_title='Valor')
        fig.show()
    
    else:
        print("Não há 'Veículo'/'Equipamento' com esse Nome!")

veiculo_Search_value(veiculoData, tabela)

In [ ]:
#--------------------------------------------Gráfico do Veículo e Motorista, GERAL(Periodo completo da planilha)

import pandas as pd
import plotly.express as px
tabela = pd.read_csv("Abastecimento_Caminhao.csv", delimiter=';', on_bad_lines='skip')
colunas_para_excluir = ["Requisição", "Hora Abast.", "Obs.", "Abast. Externo", "Data Req.", "Combustível", "Vlr. Unitário"]
colunas_existentes = [col for col in colunas_para_excluir if col in tabela.columns]
if colunas_existentes:
    tabela = tabela.drop(columns=colunas_existentes)
else:
    print("Nenhuma das colunas especificadas foi encontrada no DataFrame.")

nameReq = input("Requisitante (exemplo: 'GESSE'): ")
veiculoData = input("Veículo (exemplo: 'BT220'): ")

def gerar_graficos_separados(tabela, colunas, nameReq, veiculoData):
    filtro = tabela[(tabela['Requisitante'].str.contains(nameReq, case=False, na=False)) & 
                    (tabela['Veículo/Equip.'].str.contains(veiculoData, case=False, na=False))]
    
    if filtro.empty:
        print("Não foram encontrados dados para o requisitante e veículo fornecidos.")
        return

    for coluna in colunas:
        grafico = px.histogram(filtro, x=coluna, color="Km Rodados", title=f"Gráfico de {coluna}")
        grafico.show()
        input("Pressione Enter para continuar para o próximo gráfico...")

colunas_para_graficos = [coluna for coluna in tabela.columns if coluna != "Km Rodados"]

gerar_graficos_separados(tabela, colunas_para_graficos, nameReq, veiculoData)

In [ ]:
#--------------------------------------------Gráfico do Veículo e Motorista, De Acordo c/
#-------------------------------------------------um Determinado Intervalo de Tempo

import pandas as pd
import plotly.express as px
tabela = pd.read_csv("Abastecimento_Caminhao.csv", delimiter=';', on_bad_lines='skip')
colunas_para_excluir = ["Requisição", "Hora Abast.", "Obs.", "Abast. Externo", "Combustível"]
colunas_existentes = [col for col in colunas_para_excluir if col in tabela.columns]
if colunas_existentes:
    tabela = tabela.drop(columns=colunas_existentes)
else:
    print("Nenhuma das colunas especificadas foi encontrada no DataFrame.")
nameReq = input("Requisitante (exemplo: 'GESSE'): ")
veiculoData = input("Veículo (exemplo: 'BT220'): ")
data_inicial = input("Data inicial (exemplo: '01/04/2024'): ")
data_final = input("Data final (exemplo: '31/07/2024'): ")

def gerar_graficos_separados(tabela, colunas, nameReq, veiculoData, data_inicial, data_final):
    
    tabela['Data Req.'] = pd.to_datetime(tabela['Data Req.'], format='%d/%m/%Y')

    data_inicial = pd.to_datetime(data_inicial, format='%d/%m/%Y')
    data_final = pd.to_datetime(data_final, format='%d/%m/%Y')

    filtro = tabela[(tabela['Requisitante'].str.contains(nameReq, case=False, na=False)) & 
                    (tabela['Veículo/Equip.'].str.contains(veiculoData, case=False, na=False)) & 
                    (tabela['Data Req.'] >= data_inicial) & 
                    (tabela['Data Req.'] <= data_final)]
    
    if filtro.empty:
        print("Não foram encontrados dados para o requisitante, veículo e período fornecidos.")
        return

    for coluna in colunas:
        grafico = px.histogram(filtro, x=coluna, color="Km Rodados", title=f"Gráfico de {coluna}")
        grafico.show()
        input("Pressione Enter para continuar para o próximo gráfico...")

colunas_para_graficos = [coluna for coluna in tabela.columns if coluna != "Km Rodados"]
gerar_graficos_separados(tabela, colunas_para_graficos, nameReq, veiculoData, data_inicial, data_final)

In [ ]:
#--------------------------------------------Gráfico do Veículo e Motorista, por um Determinado Mês

import pandas as pd
import plotly.express as px

tabela = pd.read_csv("Abastecimento_Caminhao.csv", delimiter=';', on_bad_lines='skip')
colunas_para_excluir = ["Requisição", "Hora Abast.", "Obs.", "Abast. Externo", "Combustível"]
colunas_existentes = [col for col in colunas_para_excluir if col in tabela.columns]
if colunas_existentes:
    tabela = tabela.drop(columns=colunas_existentes)
else:
    print("Nenhuma das colunas especificadas foi encontrada no DataFrame.")

nameReq = input("Requisitante (exemplo: 'GESSE'): ")
veiculoData = input("Veículo (exemplo: 'BT220'): ")
mes = int(input("Mês (1-12): "))
ano = int(input("Ano (exemplo: 2023): "))

def gerar_graficos_separados(tabela, colunas, nameReq, veiculoData, mes, ano):
    tabela['Data Req.'] = pd.to_datetime(tabela['Data Req.'], format='%d/%m/%Y')

    filtro = tabela[(tabela['Requisitante'].str.contains(nameReq, case=False, na=False)) & 
                    (tabela['Veículo/Equip.'].str.contains(veiculoData, case=False, na=False)) & 
                    (tabela['Data Req.'].dt.month == mes) & 
                    (tabela['Data Req.'].dt.year == ano)]
    
    if filtro.empty:
        print("Não foram encontrados dados para o requisitante, veículo e período fornecidos.")
        return
    for coluna in colunas:
        grafico = px.histogram(filtro, x=coluna, color="Km Rodados", title=f"Gráfico de {coluna}")
        grafico.show()
        input("Pressione Enter para continuar para o próximo gráfico...")

colunas_para_graficos = [coluna for coluna in tabela.columns if coluna != "Km Rodados"]
gerar_graficos_separados(tabela, colunas_para_graficos, nameReq, veiculoData, mes, ano)